# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
import pandas as pd
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
food_db = mongo['uk_food']
food_db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food')

In [4]:
# review the collections in our database
print(food_db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = food_db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [42]:
# Find the establishments with a hygiene score of 20
query = {'scores.hygiene': 20}
fields = {'BusinessName': 1,'scores.Hygiene': 1, '_id':0}

# Capture the results to a variable
document_count = establishments.count_documents(results)

# Use count_documents to display the number of documents in the result
print(f'The establishments collection contains {document_count} document with a hygiene score of 20')

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

# review a document in the establishments collection


InvalidDocument: cannot encode object: <pymongo.cursor.Cursor object at 0x000002CDBCF64848>, of type: <class 'pymongo.cursor.Cursor'>

In [43]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(result_df))
# Display the first 10 rows of the DataFrame
result_df.head(10)

Rows in DataFrame:  0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query2 = {{'LocalAuthorityName': 'London'},{'RatingValue': {'$gte': 4}}
# fields = {'BusinessName': 1,'LocalAuthorityName': 1, 'RatingValue': 1, '_id':0}
# results = establishments.find(query, fields)

# Use count_documents to display the number of documents in the result
document_count2 = establishments.count_documents({query2})
print(f'The establishments collection contains {document_count} documents')

# Display the first document in the results using pprint
pprint(results[0:1])

SyntaxError: invalid syntax (1628751648.py, line 7)

In [ ]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(result_df))
# Display the first 10 rows of the DataFrame
result_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Determine the latitude and longitude of Penang Flavours
#food_db.establishments.find_one({"BusinessName":'Penang Flavours', 'geocode.longitude'})
lon_dict = food_db.establishments.find_one({'BusinessName':'Penang Flavours'}, {'geocode.longitude':1, '_id':0})
lon_df = pd.DataFrame(lon_dict)
Penang_longitude = lon_df.iloc[0]['geocode']

lat_dict = food_db.establishments.find_one({'BusinessName':'Penang Flavours'}, {'geocode.latitude':1, '_id':0})
lat_df = pd.DataFrame(lat_dict)
Penang_latitude = lat_df.iloc[0]['geocode']
Penang_longitude, Penang_latitude



    
    

In [33]:
# set up results search for geocodes to set as variables
results_geocodes = food_db.establishments.find_one({'BusinessName':'Penang Flavours'},
                                                   {'geocode.longitude':1, 'geocode.latitude':1,'_id':0})
results_geocodes_to_print = {'BusinessName': 1, 'geocode.longitude':1, 'geocode.latitude':1, '_id':0}
results_to_print = list(establishments.find(results_geocodes, results_geocodes_to_print).sort(sort).limit(10))
# Search within 0.01 degree on either side of the latitude and longitude.
degree_search = 0.01

# get key variables for search
for key, value in results_geocodes.items():
    latitude = value['latitude']
    longitude = value['longitude']
    
# make min/max variables for search
min_lat = latitude - degree_search
max_lat = latitude + degree_search
min_lon = longitude - degree_search
max_lon = longitude + degree_search

min_lat, max_lat, min_lon,max_lon
# # Rating value must equal 5
# rating_value = '5'

# # find the results and limit the fields to display                                                  
# query_three= {'RatingValue': rating_value,
#               'geocode.latitude':{'$gte':min_lat, '$lte':max_lat},
#               'geocode.longtitude':{'$gte':min_lon, '$lte':max_lon},
#              }
# fields = {'BusinessName': 1, 'geocode.longitude':1, 'geocode.latitude':1, 'scores.Hygiene':1, '_id':0}

# # Sort by hygiene score
# sort = [('scores.Hygiene',1)]  

# # Print the results
#   # first, the original starting point of search
# pprint(results_to_print)
# print(' '*1)
#   # second, the search answers
# results_three = list(establishments.find(query_three, fields).sort(sort).limit(10))
# pprint(results_three)

(51.480142, 51.500142, 0.07384, 0.09383999999999999)

In [ ]:
# Convert result to Pandas DataFrame
result_df = pd.DataFrame(results)
result_df.head(5)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
match_query = {'scores.hygiene': 20}

# 2. Groups the matches by Local Authority
group_query = {'$group': {'_id': {"country": "$country",
                                  "classification": "$classification"}, 
                          'count': { '$sum': 1 }
                         }
              }

# 3. Sorts the matches from highest to lowest
sort_values = {'$sort': { '_id.country': 1, 'count': -1 }}

# Print the number of documents in the result
print("Number of rows in result: ", len(results))

# Print the first 10 results
pprint(results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(result_df))
# Display the first 10 rows of the DataFrame
result_df.head(10)